# Predict rasters for a single placettes out of a parcelle

In [1]:
import warnings

warnings.simplefilter(action="ignore")

import functools
import argparse
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
from torch.utils.tensorboard import SummaryWriter
from scipy.stats import gamma
import os
import time
import torch
import torch.optim as optim
from torch.optim.lr_scheduler import StepLR
import torchnet as tnt
from sklearn.neighbors import NearestNeighbors

import gc

# from osgeo import gdal, osr  # TODO: UNCOMMENT
import torch.nn as nn
from scipy.special import digamma, polygamma

import matplotlib

# Weird behavior: loading twice in cell appears to remove an elsewise occuring error.
for i in range(2):
    try:
        matplotlib.use("TkAgg")  # rerun this cell if an error occurs.
    except:
        print("!")
import matplotlib.pyplot as plt


import pickle
from torch_scatter import scatter_max, scatter_mean

print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()

# We import from other files
import sys
sys.path.append("/home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/")
from config import args
from model.model import PointNet
from utils.useful_functions import *
from data_loader.loader import *
from utils.load_las_data import *
from model.loss_functions import *
from model.accuracy import *
from em_gamma.get_gamma_parameters_em import *
from model.train import train_full

print("Everything is imported")


print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()

False
Dataset folder in use: /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/data/
Arguments were imported in DEV mode
Everything is imported
False


## Loading single parcelle file

In [2]:
import json

In [3]:
# Create the result folder
create_new_experiment_folder(args)  # new paths are added to args
las_folder = args.las_parcelles_folder_path

# here for test we use this path
centers_dict_path = "/home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/experiments/RESULTS_3_strata/only_stratum/DEV/2021-06-02_15h02m15s/"
centers_dict_path = centers_dict_path + "parcelles_division_centers.json"

# The dictionnaries of pseudo-placettes centers are here
with open(centers_dict_path) as json_file:
    centers_dict = json.load(json_file)

13:09:50
Results folder:  /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/experiments/RESULTS_3_strata/only_stratum/DEV/2021-06-02_15h09m50s/


In [8]:
las_filename = "004000715-5-18.las"
centers = centers_dict[las_filename]

In [9]:

# las_filename = "004006880-11-1.las"

In [62]:
# Load all points from LAS file
points_nparray, xy_averages = load_single_las(las_folder, las_filename)
x_las, y_las = points_nparray[:,0], points_nparray[:,1]
subsample = True
if subsample:
    subsampling = 10000
    subset = np.random.choice(points_nparray.shape[0],size=subsampling, replace=False)
    x_las = x_las[subset]
    y_las = y_las[subset]
points_nparray = np.stack([x_las,y_las]).transpose()

In [63]:
# extract point within disk defined by center

def extract_points_within_disk(points_nparray, center, radius = 10):
    try:
        prepped_buffer = prep(center.buffer(radius))
    except:
        prepped_buffer = prep(Point(center).buffer(radius))

    multipoints = asMultiPoint(points_nparray)
    # boolean array indicating what points were contained within distance from the line
    contained = np.fromiter(map(prepped_buffer.contains, multipoints), np.bool)
    # take all rows from all_objs that are within distance from line
    contained_points = np.compress(contained, multipoints, axis=0)
    return contained_points
center = Point([911578.7 , 6316640.26])
points_within_disk = extract_points_within_disk(points_nparray, center)
points_within_disk

(437, 2)

In [ ]:
# 

# Tests with Shapely

In [30]:
# # Test from https://gis.stackexchange.com/a/192669/184486
# import shapely
# from shapely.geometry import Point
# # idx_center = 0
# # disk = Point([15,17]).buffer(10)
# # other_point = Point([15,17+9.99999])
# # other_point.within(disk)

True

In [38]:
# # Test from https://gis.stackexchange.com/a/327680/184486
# from shapely.prepared import prep
# from shapely.geometry import asMultiPoint

# radius = 10
# center = Point([15,17])
# prepped_buffer = prep(center.buffer(radius))
# two_points = [[16,17+9], [16,17+11]]
# points = asMultiPoint(two_points)
# # boolean array indicating what points were contained within distance from the line
# contained = np.fromiter(map(prepped_buffer.contains, points), np.bool)
# # take all rows from all_objs that are within distance from line
# contained_points = np.compress(contained, two_points, axis=0)
# result

array([[16, 26]])

# infer.py

In [4]:
import warnings

warnings.simplefilter(action="ignore")

import functools
import numpy as np
import pandas as pd

from sklearn.model_selection import KFold
import os
import time
import torch
import torchnet as tnt

import torch.nn as nn

import matplotlib

# Weird behavior: loading twice in cell appears to remove an elsewise occuring error.
for i in range(2):
    try:
        matplotlib.use("TkAgg")  # rerun this cell if an error occurs.
    except:
        print("!")

print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()

# We import from other files
import sys
sys.path.append("/home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/")

from config import args
from utils.useful_functions import *
from data_loader.loader import *
from utils.load_las_data import load_all_las_from_folder, open_metadata_dataframe
from model.loss_functions import *
from model.accuracy import *
from em_gamma.get_gamma_parameters_em import *
from model.train import train_full
from model.infer_utils import (
    divide_parcel_las_and_get_disk_centers,
    extract_points_within_disk,
)

print("Everything is imported")


print(torch.cuda.is_available())
np.random.seed(42)
torch.cuda.empty_cache()

# Create the result folder
create_new_experiment_folder(args)  # new paths are added to args

las_folder = args.las_parcelles_folder_path
las_filenames = os.listdir(las_folder)

centers_dict_path = os.path.join(args.stats_path, "parcelles_division_centers.json")
centers_dict = {}

for las_filename in las_filenames:
    if args.mode == "DEV":
        if las_filename != "004000715-5-18.las":
            continue
    # her we divide all parcels into plots
    grid_pixel_xy_centers, points_nparray = divide_parcel_las_and_get_disk_centers(
        args, las_folder, las_filename, save_fig_of_division=True
    )
    # print(grid_pixel_xy_centers)

    for center in grid_pixel_xy_centers:
        contained_points = extract_points_within_disk(points_nparray, center)
        print(contained_points.shape)

        # break


False
Dataset folder in use: /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/data/
Arguments were imported in DEV mode
Everything is imported
False
15:23:14
Results folder:  /home/CGaydon/Documents/LIDAR PAC/plot_vegetation_coverage/experiments/RESULTS_3_strata/only_stratum/DEV/2021-06-02_17h23m14s/
Square dimensions are 14.14m*14.14mbut we move 11.02m at a time to have 3.12m of overlap


AssertionError: 

In [6]:
points_nparray.shape

(92188, 9)

In [5]:
contained_points = extract_points_within_disk(points_nparray, center)

AssertionError: 